In [1]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import datetime
import pandas as pd
import numpy as np

In [2]:
df_api = pd.read_csv('api_foursquare.csv', index_col= 0)
df_aemet = pd.read_csv('df_aemet_final.csv', index_col= 0)
df_municipios = pd.read_csv('municipios.csv', index_col=0)

In [3]:
df_municipios=df_municipios.T

In [4]:
df_municipios

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda
ajalvir,40.5342302,-3.4807818,Ajalvir
alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
alamo-el,40.2307296,-3.9905893,El Álamo
alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...
villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


In [5]:
df_municipios.reset_index(inplace=True)

In [6]:
df_municipios.columns

Index(['index', 0, 1, 2], dtype='object')

In [7]:
df_municipios.shape

(176, 4)

In [8]:
df_municipios['index'].unique

<bound method Series.unique of 0                 acebeda-la
1                    ajalvir
2          alameda-del-valle
3                   alamo-el
4          alcala-de-henares
               ...          
171          villar-del-olmo
172    villarejo-de-salvanes
173     villaviciosa-de-odon
174    villavieja-del-lozoya
175                zarzalejo
Name: index, Length: 176, dtype: object>

In [9]:
df_municipios[1] = df_municipios[1].astype(float)

In [10]:
df_municipios[0] = df_municipios[0].astype(float)

In [11]:
df_municipios.dtypes

index     object
0        float64
1        float64
2         object
dtype: object

In [12]:
df_municipios.shape

(176, 4)

In [13]:
df_municipios['index'].duplicated().sum()

np.int64(0)

* AEMET

In [14]:
df_aemet.duplicated().sum()

np.int64(1)

In [15]:
df_aemet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7885 entries, 2024-08-29 12:00:00 to 2024-08-31 07:00:00
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cielo                  7885 non-null   object 
 1   temp._(°c)             7885 non-null   int64  
 2   sen._térmica_(°c)      7885 non-null   int64  
 3   racha_máx._(km/h)      7885 non-null   float64
 4   precipitación_(mm)     7885 non-null   float64
 5   nieve_(mm)             7885 non-null   int64  
 6   humedad_relativa_(%)   7885 non-null   int64  
 7   prob._precip.__(%)     7885 non-null   object 
 8   prob._de_nieve_(%)     7885 non-null   int64  
 9   prob._de_tormenta_(%)  7885 non-null   int64  
 10  avisos                 7885 non-null   object 
 11  dirección_viento       7885 non-null   object 
 12  velocidad_del_viento   7885 non-null   int64  
 13  localizacion_id        7885 non-null   object 
dtypes: float64(2), int64(7), obj

In [16]:
df_aemet.reset_index(inplace=True)

In [17]:
df_aemet.columns

Index(['fecha', 'cielo', 'temp._(°c)', 'sen._térmica_(°c)',
       'racha_máx._(km/h)', 'precipitación_(mm)', 'nieve_(mm)',
       'humedad_relativa_(%)', 'prob._precip.__(%)', 'prob._de_nieve_(%)',
       'prob._de_tormenta_(%)', 'avisos', 'dirección_viento',
       'velocidad_del_viento', 'localizacion_id'],
      dtype='object')

In [18]:
df_aemet['fecha'] = pd.to_datetime(df_aemet['fecha'])

In [19]:
df_aemet.nunique()

fecha                     45
cielo                     23
temp._(°c)                24
sen._térmica_(°c)         26
racha_máx._(km/h)         70
precipitación_(mm)        29
nieve_(mm)                 1
humedad_relativa_(%)      67
prob._precip.__(%)        22
prob._de_nieve_(%)         1
prob._de_tormenta_(%)     19
avisos                     2
dirección_viento           9
velocidad_del_viento      43
localizacion_id          179
dtype: int64

In [20]:
df_aemet[['nombre', 'id']] = df_aemet['localizacion_id'].str.rsplit('-', n=1, expand=True)
# quiero quedarme con el nombre y el id

In [21]:
df_aemet['id'] = df_aemet['id'].str.replace('id', '')

In [22]:
df_aemet.sample()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id,nombre,id
2887,2024-08-30 09:00:00,Cubierto,19,19,25.0,0.0,0,95,Riesgo,0,10,Riesgo,NE,18,grinon-id28066,grinon,28066


In [23]:
df_aemet_municipios = df_aemet[['nombre','id']].drop_duplicates()

In [24]:
df = df_aemet_municipios.merge(df_municipios, how='left', left_on= 'nombre', right_on= 'index')


In [25]:
df = df.drop(columns='index')

In [26]:
df 
### creada

,nombre,id,0,1,2
0,acebeda-la,28001,41.086958,-3.624399,La Acebeda
1,ajalvir,28002,40.534230,-3.480782,Ajalvir
2,alameda-del-valle,28003,40.917718,-3.843822,Alameda del Valle
3,alamo-el,28004,40.230730,-3.990589,El Álamo
4,alcala-de-henares,28005,40.481840,-3.364497,Alcalá de Henares
...,...,...,...,...,...
174,villar-del-olmo,28179,40.336447,-3.235562,Villar del Olmo
175,villarejo-de-salvanes,28180,40.168358,-3.273876,Villarejo de Salvanés
176,villaviciosa-de-odon,28181,40.357379,-3.900233,Villaviciosa de Odón
177,villavieja-del-lozoya,28182,41.006362,-3.671629,Villavieja del Lozoya


*AEMET

In [27]:
df_aemet = df_aemet.drop(columns='localizacion_id')
# cielo y riesgo

In [28]:
df_aemet.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,nombre,id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la,28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la,28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la,28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,acebeda-la,28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20,acebeda-la,28001


In [29]:
df_aemet['prob._precip.__(%)'] = df_aemet['prob._precip.__(%)'].replace('Riesgo', np.nan)


In [30]:
df_cielo = df_aemet[['cielo']].drop_duplicates().reset_index(drop=True).reset_index()

In [31]:
df_cielo.columns= ['id_cielo', 'cielo']

In [32]:
df_cielo.columns

Index(['id_cielo', 'cielo'], dtype='object')

In [33]:
df_avisos = df_aemet[['avisos']].drop_duplicates().reset_index(drop=True).reset_index()

In [34]:
df_avisos 
#creada

,index,avisos
0,0,Riesgo
1,1,Sin riesgo


In [35]:
df_avisos.columns = ['id_avisos', 'avisos']

In [36]:
df_mergeo_cielo = df_aemet.merge(df_cielo, how='inner', left_on= 'cielo', right_on= 'cielo').drop(columns='cielo')


In [37]:
df_mergeo_cielo['id_cielo'].head()

0    0
1    0
2    1
3    0
4    0
Name: id_cielo, dtype: int64

In [38]:
df_mergeo_cielo = df_mergeo_cielo.merge(df_avisos, how='inner', left_on='avisos', right_on='avisos').drop(columns='avisos')

In [39]:
df_mergeo_cielo.head()

,fecha,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,nombre,id,id_cielo,id_avisos
0,2024-08-29 12:00:00,21,21,25.0,0.0,0,69,95,0,75,S,7,acebeda-la,28001,0,0
1,2024-08-29 13:00:00,24,24,29.0,0.0,0,57,NaN,0,75,SE,12,acebeda-la,28001,0,0
2,2024-08-29 14:00:00,23,23,33.0,0.0,0,59,100,0,80,S,14,acebeda-la,28001,1,0
3,2024-08-29 15:00:00,23,23,35.0,0.0,0,55,NaN,0,80,S,17,acebeda-la,28001,0,0
4,2024-08-29 16:00:00,21,21,34.0,0.0,0,65,NaN,0,80,S,20,acebeda-la,28001,0,0


In [40]:
df_mergeo_cielo= df_mergeo_cielo.drop(columns='nombre')

In [41]:
df_mergeo_cielo.head()

,fecha,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,id,id_cielo,id_avisos
0,2024-08-29 12:00:00,21,21,25.0,0.0,0,69,95,0,75,S,7,28001,0,0
1,2024-08-29 13:00:00,24,24,29.0,0.0,0,57,NaN,0,75,SE,12,28001,0,0
2,2024-08-29 14:00:00,23,23,33.0,0.0,0,59,100,0,80,S,14,28001,1,0
3,2024-08-29 15:00:00,23,23,35.0,0.0,0,55,NaN,0,80,S,17,28001,0,0
4,2024-08-29 16:00:00,21,21,34.0,0.0,0,65,NaN,0,80,S,20,28001,0,0


In [42]:
df_aemet.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,nombre,id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la,28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,NaN,0,75,Riesgo,SE,12,acebeda-la,28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la,28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,NaN,0,80,Riesgo,S,17,acebeda-la,28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,NaN,0,80,Riesgo,S,20,acebeda-la,28001


In [43]:
df_mergeo_cielo = df_mergeo_cielo.rename(columns={'id':'id_municipio'})

In [44]:
df_mergeo_cielo.head()

,fecha,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,id_municipio,id_cielo,id_avisos
0,2024-08-29 12:00:00,21,21,25.0,0.0,0,69,95,0,75,S,7,28001,0,0
1,2024-08-29 13:00:00,24,24,29.0,0.0,0,57,NaN,0,75,SE,12,28001,0,0
2,2024-08-29 14:00:00,23,23,33.0,0.0,0,59,100,0,80,S,14,28001,1,0
3,2024-08-29 15:00:00,23,23,35.0,0.0,0,55,NaN,0,80,S,17,28001,0,0
4,2024-08-29 16:00:00,21,21,34.0,0.0,0,65,NaN,0,80,S,20,28001,0,0


*API

In [45]:
df_api.head()

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166


In [78]:
df_api['category'].value_counts()

category
Park                                                738
Monument                                            126
TV Station                                           24
Bridge                                                6
Film Studio                                           3
Castle                                                3
Fountain                                              3
Beach                                                 2
Palace                                                1
Print, TV, Radio and Outdoor Advertising Service      1
Name: count, dtype: int64

In [46]:
df_categorias = df_api[['category']].drop_duplicates().reset_index(drop=True).reset_index()

In [47]:
df_categorias

,index,category
0,0,Park
1,1,Monument
2,2,TV Station
3,3,Castle
4,4,Film Studio
5,5,Palace
6,6,Fountain
7,7,Bridge
8,8,Beach
9,9,"Print, TV, Radio and Outdoor Advertising Service"


In [48]:
df_categorias = df_categorias.rename(columns={'index':'id_category'})

In [49]:
df_mergeo_api_places = df_api.merge(df_categorias, how='inner', left_on='category', right_on='category').drop(columns='category')

In [50]:
df_mergeo_api_places.head()

,municipio,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_category
0,ajalvir,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,0
1,ajalvir,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773,0
2,ajalvir,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835,0
3,alamo-el,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,0
4,alamo-el,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166,0


In [51]:
df_mergeo_api_places=  df_mergeo_api_places.merge(df[['nombre', 'id']], how='inner', left_on='municipio', right_on='nombre').drop(columns=['municipio', 'nombre'])

In [52]:
# df_mergeo_api_places = df_mergeo_api_places.rename(columns=)

In [53]:
df_mergeo_api_places = df_mergeo_api_places.rename(columns={'id':'id_municipio'})

In [54]:
df_mergeo_api_places

,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_category,id_municipio
0,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,0,28002
1,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773,0,28002
2,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835,0,28002
3,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,0,28004
4,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166,0,28004
...,...,...,...,...,...,...,...,...,...,...
902,51ffcb685019c1bb402374fd,LikelyOpen,1043.0,/v3/places/51ffcb685019c1bb402374fd,Matadero de Villaviciosa CQB ARISOFT,28670 Villaviciosa de Odón Comunidad de Madrid,40.351365,-3.890769,0,28181
903,4c7a6f88566db60c43e7470e,LikelyOpen,1626.0,/v3/places/4c7a6f88566db60c43e7470e,Parque del Castillo,"Avenida del Castillo, 28670 Villaviciosa de Od...",40.357793,-3.881113,0,28181
904,5834bfa364f54721bc08814d,Unsure,747.0,/v3/places/5834bfa364f54721bc08814d,Discoteca Doblón,"Avenida Principe de Asturias, 94, Villaviciosa...",40.360011,-3.908360,1,28181
905,4ed124af5c5c9528fa2230de,LikelyOpen,1577.0,/v3/places/4ed124af5c5c9528fa2230de,Pinar Zarzalejo,Pinar,40.540691,-4.165816,0,28183


In [55]:
try:
    conexion = psycopg2.connect(
    database = 'Productora',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [56]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion = """ 
    CREATE TABLE IF NOT EXISTS municipios (
    nombre varchar(100),
    id_municipio int PRIMARY KEY,
    latitud float,
    longitud float,
    nombre_propio varchar(100));

"""

cursor.execute(query_creacion)
conexion.commit()
#recuerda que cambiaste el noombre de id a id_municipio

In [57]:
lista_tuplas1 = []
for fila in df.values:
    lista_tuplas1.append(tuple(fila))
lista_tuplas1[0]

# [tuple(fila) for fila in df.values]

('acebeda-la', '28001', 41.0869583, -3.624399, 'La Acebeda')

In [58]:
query_insercion = 'INSERT INTO municipios (nombre, id_municipio, latitud, longitud, nombre_propio) VALUES (%s, %s, %s, %s, %s)' # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas1:
    cursor.execute(query_insercion, valores)
conexion.commit()

In [59]:
cursor = conexion.cursor() #para crear la tabla desde aquí
query_creacion_cielo = """ 
    CREATE TABLE IF NOT EXISTS cielo (
    id_cielo int PRIMARY KEY,
    cielo varchar(100));

"""

cursor.execute(query_creacion_cielo)
conexion.commit()

In [60]:
lista_tuplas2 = []
for fila in df_cielo.values:
    lista_tuplas2.append(tuple(fila))
lista_tuplas2[0]

(0, 'Nubes altas')

In [61]:
query_creacion_cielo = 'INSERT INTO cielo (id_cielo, cielo) VALUES (%s, %s)' # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas2:
    cursor.execute(query_creacion_cielo, valores)
conexion.commit()

In [62]:
cursor = conexion.cursor()
query_creacion_categorias = """
    CREATE TABLE IF NOT EXISTS categorias (
    id_categoria int primary key,
    category varchar(100));
"""
cursor.execute(query_creacion_categorias)
conexion.commit()

In [63]:
lista_tuplas3 = []
for fila in df_categorias.values:
    lista_tuplas3.append(tuple(fila))
lista_tuplas3[0]

(0, 'Park')

In [64]:
query_creacion_categorias = 'INSERT INTO categorias (id_categoria, category) VALUES (%s, %s)' # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas3:
    cursor.execute(query_creacion_categorias, valores)
conexion.commit()

In [65]:
cursor = conexion.cursor()
query_creacion_avisos = """
    CREATE TABLE IF NOT EXISTS avisos (
    id_avisos int primary key,
    avisos varchar(100));
"""
cursor.execute(query_creacion_avisos)
conexion.commit()

In [66]:
lista_tuplas4 = []
for fila in df_avisos.values:
    lista_tuplas4.append(tuple(fila))
lista_tuplas4

[(0, 'Riesgo'), (1, 'Sin riesgo')]

In [67]:
query_creacion_avisos = 'INSERT INTO avisos (id_avisos, avisos) VALUES (%s, %s)' # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas4:
    cursor.execute(query_creacion_avisos, valores)
conexion.commit()

In [68]:
df_mergeo_cielo.dtypes

fecha                    datetime64[ns]
temp._(°c)                        int64
sen._térmica_(°c)                 int64
racha_máx._(km/h)               float64
precipitación_(mm)              float64
nieve_(mm)                        int64
humedad_relativa_(%)              int64
prob._precip.__(%)               object
prob._de_nieve_(%)                int64
prob._de_tormenta_(%)             int64
dirección_viento                 object
velocidad_del_viento              int64
id_municipio                     object
id_cielo                          int64
id_avisos                         int64
dtype: object

In [69]:
query_creacion_AEMET = """ 
    CREATE TABLE IF NOT EXISTS AEMET (
    fecha DATE,
    temperatura INT NOT NULL,
    sen_termica_c INT NOT NULL,
    racha_max_kmh FLOAT,
    precipitacion_mm FLOAT,
    nieve_mm INT,
    humedad_relativa INT,
    prob_precip FLOAT, 
    prob_de_nieve INT,
    prob_de_tormenta INT,
    direccion_viento VARCHAR(10),
    velocidad_del_viento INT,
    id_municipio INT, 
    id_cielo INT,
    id_avisos INT,
    CONSTRAINT fk_municipio FOREIGN KEY (id_municipio)
        REFERENCES municipios(id_municipio),
    CONSTRAINT fk_cielo FOREIGN KEY (id_cielo)
        REFERENCES cielo(id_cielo),
    CONSTRAINT fk_avisos FOREIGN KEY (id_avisos)
        REFERENCES avisos(id_avisos)
);
"""
# cursor.execute(query_creacion_AEMET)
# conexion.commit()

In [70]:
query_creacion_AEMET = """ 
    CREATE TABLE IF NOT EXISTS AEMET (
    fecha date,
    temperatura int not null,
    sen_termica_c int not null,
    racha_max_kmh float,
    precipitacion_mm float,
    nieve_mm int,
    humedad_relativa int,
    prob_precip_ varchar(100), 
    prob_de_nieve_ int,
    prob_de_tormenta_ int,
    direccion_viento varchar(10),
    velocidad_del_viento int,
    id_municipio int, 
    id_cielo int,
    id_avisos int,
    foreign key (id_municipio)
        references municipios(id_municipio),
    foreign key (id_cielo)
        references cielo(id_cielo),
    foreign key (id_avisos)
        references avisos(id_avisos));

    """

cursor.execute(query_creacion_AEMET)
conexion.commit()


In [71]:
# lista_tuplas5 = []
lista_tuplas5 = []
for fila in df_mergeo_cielo.values:
    lista_tuplas5.append(tuple(fila))
lista_tuplas5[0]

(Timestamp('2024-08-29 12:00:00'),
 21,
 21,
 25.0,
 0.0,
 0,
 69,
 '95',
 0,
 75,
 'S',
 7,
 '28001',
 0,
 0)

In [72]:
query_creacion_AEMET = 'INSERT INTO AEMET (fecha, temperatura, sen_termica_c, racha_max_kmh, precipitacion_mm, nieve_mm, humedad_relativa, prob_precip_, prob_de_nieve_, prob_de_tormenta_, direccion_viento, velocidad_del_viento, id_municipio, id_cielo, id_avisos) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

for valores in lista_tuplas5:
    cursor.execute(query_creacion_AEMET, valores)
conexion.commit()

In [73]:
df_mergeo_api_places.dtypes

fsq_id            object
closed_bucket     object
distance         float64
link              object
name              object
address           object
latitude         float64
longitude        float64
id_category        int64
id_municipio      object
dtype: object

In [74]:
query_creacion_lugares = """ 
    CREATE TABLE IF NOT EXISTS lugares (
    fsq_id varchar(100),
    closed_bucket varchar(100),
    distance int,
    link varchar(100),
    name varchar(100),
    address varchar(300),
    latitude float, 
    longitude float,
    id_category int,
    id_municipio int,
    foreign key (id_category)
        references categorias(id_categoria),
    foreign key (id_municipio)
        references municipios(id_municipio));

    """

cursor.execute(query_creacion_lugares)
conexion.commit()
# recuerda que cambiaste id_municipio a int cuando era varchar(100)
#recuerda que quitaste fsq de primary key
# cambiaste address a 300

In [75]:
lista_tuplas6 = []
for fila in df_mergeo_api_places.values:
    lista_tuplas6.append(tuple(fila))
lista_tuplas6[0]

('4d3c6c5b84d46ea87dddfd5c',
 'LikelyOpen',
 563.0,
 '/v3/places/4d3c6c5b84d46ea87dddfd5c',
 'Diverjungla',
 'Calle Segovia, 20 (Pol. Ind. Donada), 28864 Ajalvir Comunidad de Madrid',
 40.529921,
 -3.484341,
 0,
 '28002')

In [76]:
query_creacion_lugares = 'INSERT INTO lugares (fsq_id, closed_bucket, distance, link, name, address, latitude, longitude, id_category, id_municipio) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)' # insertar valores %s es como un formato. como un argumento
for valores in lista_tuplas6:
    cursor.execute(query_creacion_lugares, valores)
conexion.commit()

In [77]:
df.head()

,nombre,id,0,1,2
0,acebeda-la,28001,41.086958,-3.624399,La Acebeda
1,ajalvir,28002,40.534230,-3.480782,Ajalvir
2,alameda-del-valle,28003,40.917718,-3.843822,Alameda del Valle
3,alamo-el,28004,40.230730,-3.990589,El Álamo
4,alcala-de-henares,28005,40.481840,-3.364497,Alcalá de Henares
